In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import Window as wn

In [2]:
spark = SparkSession.builder.master("local[2]").appName("Min MAx Salary").getOrCreate()

In [3]:
emp_data = [
    ('Siva',1,30000),
    ('Ravi',2,40000),
    ('Prasad',1,50000),
    ('Arun',1,30000),
    ('Sai',2,20000)
]
emp_schema = "emp_name string , dep_id int , salary long"
df = spark.createDataFrame(data = emp_data , schema = emp_schema)
df.show()

+--------+------+------+
|emp_name|dep_id|salary|
+--------+------+------+
|    Siva|     1| 30000|
|    Ravi|     2| 40000|
|  Prasad|     1| 50000|
|    Arun|     1| 30000|
|     Sai|     2| 20000|
+--------+------+------+



In [17]:
part_df = df.withColumn("DRN",FS.dense_rank().over(wn.partitionBy("dep_id").orderBy(FS.col("salary").desc()))).\
withColumn("MAx",FS.max("salary").over(wn.partitionBy("dep_id"))).\
withColumn("MIN",FS.min("salary").over(wn.partitionBy("dep_id")))
part_df.show()

+--------+------+------+---+-----+-----+
|emp_name|dep_id|salary|DRN|  MAx|  MIN|
+--------+------+------+---+-----+-----+
|  Prasad|     1| 50000|  1|50000|30000|
|    Siva|     1| 30000|  2|50000|30000|
|    Arun|     1| 30000|  2|50000|30000|
|    Ravi|     2| 40000|  1|40000|20000|
|     Sai|     2| 20000|  2|40000|20000|
+--------+------+------+---+-----+-----+



In [14]:
part_df.filter(FS.col("DRN") == 1).show()

+--------+------+------+---+
|emp_name|dep_id|salary|DRN|
+--------+------+------+---+
|  Prasad|     1| 50000|  1|
|    Ravi|     2| 40000|  1|
+--------+------+------+---+



In [18]:
# part_df.groupBy("dep_id").agg(
#     FS.when(FS.max(FS.col('DRN')) == 1,FS.col('emp_name')).alias('min'),
#      FS.when(FS.min(FS.col('DRN')) == FS.col('dep_id'),FS.col('emp_name')).alias('max')).show()